![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to capture video from a MIPI device, processes it through 2D convolution filter accelerator  and display the output on a monitor using a DRM/KMS display device. This notebook uses the GStreamer multimedia framework. In addition, the memory bandwidth is measured and plotted in a parallel notebook.

The display device uses the Xilinx DRM/KMS driver. A video mixer supports alpha blending of multiple layers (also called planes). 
The plane formats are fixed and configured as follows:
* 4 BG24 planes (IDs: 34-37)
* 4 YUY2 planes (IDs: 38-41)
* 1 ARGB plane (ID: 42) - this is the primary plane used for setting the CRTC resolution

The video mixer is connected to an HDMI encoder which drives the display. Both video mixer and HDMI encoder are implemented inside the FPGA.

The 2D filter has a fixed kernel size of 3x3. It operates on the luma channel of a YUY2 image; The PL 2D filter performs the chroma loopback inside the kernel itself. In addition, the PL version has run-time programmable kernel coefficients and presets as well as well image dimensions.

For more information on the 2D convolution filter operation, see here: https://en.wikipedia.org/wiki/Kernel_(image_processing).

The various 2D filter accelerators are integrated into Gstreamer using the Vitis Video Analytics SDK (VVAS) which provides a set of plugins and an abstraction layer on top of the Xilinx run-time (XRT). For more information, see here: https://xilinx.github.io/VVAS/.

2D filter is implemented in the PL using the Vitis Vision libraries and high-level synthesis (HLS)

The video pipeline is composed of the following GStreamer elements:
* The ``xlnxvideosrc`` element is used to capture video from a V4L2 device
* The ``vvas_xfilter`` element is used to implement a 2D convolution filter
* The ``perf`` element is used to measure and print the frame rate
* The ``kmssink`` element is used to display video on a monitor using the DRM/KMS kernel subsystem

The ``vmk180-trd-apm`` notebook is executed in parallel to this notebook to measure and plot the memory bandwidth of the live video pipeline.

In this notebook, you will:
1. Create a GStreamer video pipeline that captures video from a V4L2 device and displays the video on a monitor using DRM/KMS.
2. The vvas_xfilter element is used to wrap the 2D convolution filter accelerator library (Optional)
3. Gstreamer pipeline with and without filter is created using the configuration parameter.
4. Run the ``trd-apm`` notebook to measure and plot the memory bandwidth while the video pipeline is running.
5. Create a GStreamer pipeline graph and view it inside this notebook.


# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import pydot
import subprocess
import sys
import time
import gi
import os
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
from gi.repository import GObject, GLib, Gst, GstApp

This is the VMK180 TRD notebook 2 (nb2).

In [ ]:
nb = "nb2"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environement variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Initialize the GStreamer library. Optionally enable debug (default off) and set the debug level.

In [ ]:
Gst.init(None)
Gst.debug_set_active(False)
Gst.debug_set_default_threshold(1)

# 3. Run the APM Notebook to Plot the Memory Bandwidth

Open the ``vmk180-trd-apm.ipynb`` notebook from the *File Browser* in a new tab. Execute the notebook by selecting *Run -> Run All Cells* from the Jupyter Lab menu bar. In section 4 of the APM notebook, a horizontal bar graph is shown that plots the currently consumed memory bandwidth split out by different AXI ports. For more information, read the APM notebook tutorial.

Once you see the graph, right-click the graph and select *Create New View for Output*. This will create a new window/tab with just the graph. Now re-arrange the window by dragging it to the the right side of the screen so it shows side-by-side with the notebook window (see screenshot below).

![APM Plot](images/apm-plot-nb3.jpg "APM Plot")

Switch tabs back to the nb2 notebook and follow the steps below. Once the video pipeline is running, you will notice the bar graph will be updated live with the measured memory bandwidth numbers in Gbps.

**Note:** You can keep the memory bandwith output view open while switching between notebooks. There is no need to restart the APM notebook.

Validate the correct Vitis Overlay is available in the platform for this notebook.

In [ ]:
device = "0000:00:00.0"
def xbutil_program_xclbin():
    xclbin = "/boot/binary_container_1.xclbin"
    if os.path.exists(xclbin):
        subprocess.run(['xbutil', 'program', '-d', device, '-u', xclbin], check=True)

def xbutil_query_cu(cu):
    proc = subprocess.run(['xbutil', 'examine', '-d', device], capture_output=True, encoding='utf8')
    for line in proc.stdout.splitlines():
        if cu in line:
            return
    raise Exception("Unable to find compute unit \'" + cu + "\'. Make sure the correct Vitis overlay is used.")

xbutil_program_xclbin()
xbutil_query_cu("filter2d_pl_accel")
#xbutil_query_cu("filter2d_aie_accel")

# 4. Create and Configure the GStreamer Elements

Create the ``xlnxvideosrc`` element and set some properties:

For MIPI,I/O mode is set to DMABUF (https://www.kernel.org/doc/html/v4.16/driver-api/dma-buf.html) which allows sharing of video buffers in 0-copy fashion between the source and sink elements.

In [ ]:
src_type = "mipi"
io_mode = "dmabuf"

src = Gst.ElementFactory.make("xlnxvideosrc")
src.set_property("io-mode", io_mode)
src.set_property("src-type", src_type)

Create a caps filter element to set the desired resolution (width and height) and format. The caps filter is configured to parse the mentioned properties from a string.

``mipi`` as source type, the maximum supported resolution is 3840x2160 (4K) at 60 fps. Note that the connected monitor also needs to support this resolution, otherwise the pipeline will fail during caps negotiation (see modeprint output below).

In [ ]:
res_dict = {
    "720p" : ("1280", "720"),
    "1080p" : ("1920", "1080"),
    "2160p" : ("3840", "2160")
}
res = "2160p" # Change the resolution string to 720p, 1080p, or 2160p (mipi only)
width = res_dict[res][0]
height = res_dict[res][1]
print("Selected resolution: " + width + "x" + height)

fmt = "YUY2"
fps = "60/1"
caps = Gst.ElementFactory.make("capsfilter")
cap = Gst.Caps.from_string("video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt + ", framerate=" + fps)
caps.set_property("caps", cap)

In [ ]:
add_filter = '0' # By default pipeline is created without filter, change this variable to 1 to add filter.

In [ ]:
if add_filter == '1':
    jsondir = "/usr/share/vvas/vmk180-trd/"
    filter_kernels = ["PL", "SW"]
    filter_kernel = filter_kernels[0] # Change filter kernel to PL, 
    print("Selected filter2d kernel: " + filter_kernel)
    if filter_kernel == "PL":
        jfile = jsondir + "kernel_xfilter2d_pl.json"
    else: # filter_kernel == "SW"
        jfile = jsondir + "kernel_xfilter2d_sw.json"

    filter2d = Gst.ElementFactory.make("vvas_xfilter")
    filter2d.set_property("kernels-config", jfile)

The PL filter2d has two ways of programming the filter coefficients:

by setting the filter_preset which translates to a set of coefficients
by setting the filter_coefficients directly

First, we look at how to set the filter_preset parameter. The below command returns a list of supported presets to choose from.

In [ ]:
if add_filter == '1':
    plist = [
        "blur",
        "edge",
        "horizontal edge",
        "vertical edge",
        "emboss",
        "horizontal gradient",
        "vertical gradient",
        "identity",
        "sharpen",
        "horizontal sobel",
        "vertical sobel",
        "custom"
    ]

    def print_presets():
        print("Supported filter presets:\n")
        print('\n'.join(plist) + '\n')
    
    #print_presets()

Set the filter_preset parameter to one of the supported values listed above e.g. "horizontal sobel" by passing the json string via the dynamic-config element property. The kernel library reads the preset and programs the filter coefficients behind the scene.

In [ ]:
if add_filter == '1' :
    def set_preset(val):
        if val in plist:
            jstring = '{ "filter_preset" : "' +  val + '" }'
            print(jstring)
            filter2d.set_property("dynamic-config", jstring)
        else:
            print("excep")
            raise Exception("Unsupported filter preset \'" + val + "\'")

    set_preset("identity")

The second way is to explicitly program the coefficients via the filter_coefficients parameter. Uncomment the last line in the next cell to program the coefficients via the dynamic-config element property.

The filter coefficients are a 3x3 matrix of short int values. The default values in the below code snippet correspond to the identity matrix which results is a simple passthrough. The identity coefficients are as follows:

 0  0  0
 0  1  0
 0  0  0
To match the coefficients for "horizontal sobel", use the following matrix:

 1  2  1
 0  0  0
-1 -2 -1

In [ ]:
if add_filter == '1':
    def set_coeff(val):
        jstring = '{ "filter_coefficients" : ' + val + ' }'
        #print(jstring)
        filter2d.set_property("dynamic-config", jstring)

    #set_coeff("[[0, 0, 0], [0, 1, 0], [0, 0, 0]]")

Create the ``perf`` element which is used to measure and print the frame rate while the video pipeline is running.

In [ ]:
perf = Gst.ElementFactory.make("perf")

The display driver creates a DRM device node with the module name ``xlnx``.

List information about the DRM device by passing the module name to the ``modeprint`` utility.

In [ ]:
!modeprint xlnx

Create the ``kmssink`` element and set some properties:
* Set the ``driver-name`` property to the Xilinx DRM driver name ``xlnx``.
* Set the ``plane-id`` property to the ID value of the target plane. The default value 34 is set to the first YUY2 plane.
* Set the ``fullscreen-overlay`` property to ``False`` to keep the CRTC set to the native display resolution.
* Set the ``render-rectangle`` property to a quadruple consisting of x-offset, y-offset, width, and height. The render-rectangle allows moving a plane position on the display.

In [ ]:
driver_name = "xlnx"
plane_id = 38
xoff = 0 # Change this value to move the plane position in the x-direction
yoff = 0 # Change this value to move the plane position in the y-direction
width = int(width, 10)
height = int(height, 10)
render_rectangle = Gst.ValueArray((xoff, yoff, width, height))

sink = Gst.ElementFactory.make("kmssink")
sink.set_property("driver-name", driver_name)
sink.set_property("plane-id", plane_id)
sink.set_property("render-rectangle", render_rectangle)
#sink.set_property("sync", False)

# Uncomment the below code to read back the newly set property values
#print("sink properties: ")
#print("driver-name: " + str(sink.get_property("driver-name")))
#print("plane-id: " + str(sink.get_property("plane-id")))


# 5. Create and Run the GStreamer Pipeline

Create the pipeline, add all elements, and link them together.

In [ ]:
pipeline = Gst.Pipeline.new(nb)

pipeline.add(src)
pipeline.add(caps)
pipeline.add(perf)
if add_filter == '1':
    pipeline.add(filter2d) # Create the filter2d element
pipeline.add(sink)

src.link(caps)
if add_filter == '1':
    caps.link(filter2d) # Link the fiter2d element's sink pad to the caps element 
    filter2d.link(perf)
else:
    caps.link(perf)
perf.link(sink);
print(pipeline)

The ``bus_call`` function listens on the bus for ``EOS``, ``INFO`` and ``ERROR`` events. In case of ``EOS`` or ``ERROR``, stop the pipeline (set to ``NULL`` state) and quit the main loop. 

For ``INFO`` and ``ERROR`` events, parse and print the info/error message. The ``perf`` element generates ``INFO`` events with the measured frame rate.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.INFO:
        err, info = message.parse_info()
        sys.stderr.write("Info: %s\n" % info)
        clear_output(wait=True)
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video will be displayed on the monitor. The frame rate will be printed and updated below the code cell.

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    Gst.deinit()
    pass

# 7. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that demonstrates how to capture video from a V4L2 device and display it on a monitor
2. Plot the live memory bandwidth by running the APM notebook in parallel
3. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>